In [19]:
import yfinance as yf
pltr = yf.Ticker("^GSPC")
pltr = pltr.history(period="max")
pltr.index
##pltr.plot.line(y="Close",use_index=True)

pltr["Tomorrow"] = pltr["Close"].shift(-1)
pltr["Target"] = (pltr["Tomorrow"] > pltr["Close"]).astype(int)
print(pltr.size)

##pltr




216846


In [20]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=100, min_samples_split=20, random_state=1)
train = pltr.iloc[:-100]
test = pltr.iloc[-100:]
predictors = ["Close","Volume", "Open", "High", "Low"]
model.fit(train[predictors], train["Target"])
from sklearn.metrics import precision_score
preds = model.predict(test[predictors])
import pandas as pd
preds = pd.Series(preds, index=test.index)
precision_score(test["Target"], preds)




0.5285714285714286

In [21]:
def predict(train,test, predictors,model):
    model.fit(train[predictors],train["Target"])
    preds = model.predict(test[predictors])
    preds = pd.Series(preds,index=test.index, name="Predictions")
    combined = pd.concat([test["Target"],preds],axis=1)
    return combined
def backtest(data,model,predictors,start=100, step=100):
    all_predictions = []
    for i in range(start, data.shape[0], step):
        train = data.iloc[0:i].copy()
        test = data.iloc[i:(i+step)].copy()
        predictions = predict(train,test,predictors,model)
        all_predictions.append(predictions)
    return pd.concat(all_predictions)
predictions = backtest(pltr,model,predictors)
predictions["Predictions"].value_counts()

KeyboardInterrupt: 

In [13]:
precision_score(predictions["Target"],predictions["Predictions"])

0.546448087431694

In [6]:
predictions["Target"].value_counts()/predictions.shape[0]

Target
1    0.508418
0    0.491582
Name: count, dtype: float64

In [14]:
horizons = [2,5,60,120]
new_predictors =[]
for horizon in horizons:
    rolling_averages = pltr.rolling(horizon).mean()
    ratio_column = f"Close_Ratio_{horizon}"
    pltr[ratio_column] = pltr["Close"] / rolling_averages["Close"]
    
    trend_column = f"Trend_{horizon}"
    pltr[trend_column] = pltr.shift(1).rolling(horizon).sum()["Target"]
    
    new_predictors += [ratio_column, trend_column]
pltr

C:\Users\mikes\AppData\Local\Temp\ipykernel_23760\4175276425.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pltr[ratio_column] = pltr["Close"] / rolling_averages["Close"]
C:\Users\mikes\AppData\Local\Temp\ipykernel_23760\4175276425.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pltr[trend_column] = pltr.shift(1).rolling(horizon).sum()["Target"]


,Open,High,Low,Close,Volume,Dividends,Stock Splits,Tomorrow,Target,Close_Ratio_2,Trend_2,Close_Ratio_5,Trend_5,Close_Ratio_60,Trend_60,Close_Ratio_120,Trend_120
Date,,,,,,,,,,,,,,,,,
2021-03-24 00:00:00-04:00,23.299999,23.485001,21.750000,21.879999,69271900,0.0,0.0,22.580000,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-03-25 00:00:00-04:00,21.200001,22.799999,21.049999,22.580000,74592600,0.0,0.0,22.580000,0,1.015745,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-03-26 00:00:00-04:00,22.629999,23.049999,21.760000,22.580000,57916400,0.0,0.0,21.730000,0,1.000000,1.0,NaN,NaN,NaN,NaN,NaN,NaN
2021-03-29 00:00:00-04:00,22.480000,22.500000,21.389999,21.730000,57647400,0.0,0.0,22.040001,1,0.980817,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2021-03-30 00:00:00-04:00,21.360001,22.090000,20.860001,22.040001,52039700,0.0,0.0,23.290001,1,1.007083,1.0,0.994495,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-11-21 00:00:00-05:00,21.110001,21.850000,19.719999,19.799999,106643300,0.0,0.0,19.690001,0,0.962567,1.0,0.976813,3.0,1.191431,35.0,1.215463,66.0
2023-11-22 00:00:00-05:00,20.260000,20.440001,19.330000,19.690001,68326000,0.0,0.0,19.200001,0,0.997215,0.0,0.973885,2.0,1.179737,34.0,1.205523,66.0
2023-11-24 00:00:00-05:00,19.379999,19.600000,18.930000,19.200001,39800400,0.0,0.0,19.080000,0,0.987400,0.0,0.955034,2.0,1.147090,33.0,1.173152,65.0


In [17]:
pltr = pltr.dropna()
model = RandomForestClassifier(n_estimators=100, min_samples_split=20, random_state=1)
def predict(train,test, predictors,model):
    model.fit(train[predictors],train["Target"])    
    preds = model.predict_proba(test[predictors])[:,1]
    preds[preds >= 0.6] = 1
    preds[preds < 0.6] = 0
    preds = pd.Series(preds,index=test.index, name="Predictions")
    combined = pd.concat([test["Target"],preds],axis=1)
    return combined

predictions = backtest(pltr,model,new_predictors)
predictions["Predictions"].value_counts()

Predictions
0.0    429
1.0     27
Name: count, dtype: int64

In [18]:
precision_score(predictions["Target"],predictions["Predictions"])

0.4444444444444444